In [12]:
# ! pip install slack_sdk

In [11]:
import requests
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import xml.etree.ElementTree as ET
from slack_sdk import WebClient
import os
import re

SLACK_TOKEN = os.environ["SLACK_TOKEN"]
CHANNEL = "#events-rss"  # or use channel ID

# Pacific timezone
PACIFIC = ZoneInfo("America/Los_Angeles")
tomorrow = datetime.now(PACIFIC).date() + timedelta(days=1)

client = WebClient(token=SLACK_TOKEN)

# Fetch the XML feed
URL = "https://calendar.usc.edu/calendar/1.xml"
response = requests.get(URL)
response.raise_for_status()
xml_data = response.content
root = ET.fromstring(xml_data)

events_today = []

for item in root.findall(".//item"):
    title = item.findtext("title") or "No title"
    link = item.findtext("link") or ""
    pub_date = item.findtext("{http://purl.org/dc/elements/1.1/}date") or item.findtext("pubDate")

    if pub_date:
        try:
            # Try ISO format first
            if "T" in pub_date:
                event_date = datetime.fromisoformat(pub_date).date()
            else:
                # Fallback for <pubDate> style (e.g., "Tue, 04 Nov 2025 00:00:00 -0800")
                event_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z").date()

            if event_date == tomorrow:
                # Remove date prefix from title
                clean_title = re.sub(r"^[A-Za-z]{3,9}\s+\d{1,2},\s+\d{4}:\s*", "", title)
                # Embed link in the title while keeping bullet
                events_today.append(f"• <{link}|{clean_title}>")

        except Exception as e:
            print("Bad date format:", pub_date, e)

# Prepare and send Slack message
if events_today:
    formatted_date = tomorrow.strftime("%b %d, %Y")
    border = "────────────────────────────────────────────────────────────"
    message = (
        f"*📅 USC Events Happening Tomorrow ({formatted_date}):*\n\n"
        + "\n"
        + "\n".join(f"• {e}" for e in events_today)
        + "\n"
        + f"\n\n{border}"
        + "\n"
    )

    resp = client.chat_postMessage(channel=CHANNEL, text=message)

    if resp.get("ok"):
        print("Slack message sent successfully ✅")
    else:
        print("Slack message failed:", resp)

else:
    print("No events found for tomorrow.")


Slack message sent successfully ✅
